# 🔌 에이전트에 미들웨어 추가하기

> **개요**  
> 에이전트에 미들웨어를 추가하여 로깅, 보안 및 기타 공통 관심사를 위해 에이전트 상호 작용을 가로채고 수정하는 방법을 알아봅니다.

> **📖 참고 자료**  
> 미들웨어에 대한 자세한 내용은 [미들웨어 문서](https://learn.microsoft.com/en-us/agent-framework/user-guide/agents/agent-middleware?pivots=programming-language-python)를 참조하세요.

---

## ✍️ 미들웨어 작성하기

간단한 미들웨어를 작성하여 에이전트의 동작을 확장할 수 있습니다. 예를 들어, 에이전트가 응답을 생성하기 전에 로그를 기록하는 미들웨어를 추가할 수 있습니다.

### 📌 Agent 기반 미들웨어

에이전트 기반 미들웨어는 에이전트 실행을 가로채고 수정합니다. `AgentRunContext`를 사용하며, 다음 속성을 제공합니다:

| 속성 | 설명 |
|------|------|
| **agent** | 호출되는 에이전트 |
| **messages** | 대화 내 채팅 메시지 목록 |
| **is_streaming** | 응답이 스트리밍 중인지를 나타내는 Boolean 값 |
| **metadata** | 미들웨어 간에 추가 데이터를 저장하기 위한 딕셔너리 |
| **result** | 에이전트의 응답 (수정 가능) |
| **terminate** | 추가 처리를 중지하는 플래그 |
| **kwargs** | 에이전트 run 메서드에 전달되는 추가 키워드 인수 |

In [ ]:
from agent_framework import AgentRunContext
from typing import Callable, Awaitable

async def logging_agent_middleware(
    context: AgentRunContext,
    next: Callable[[AgentRunContext], Awaitable[None]],
) -> None:
    """ 에이전트 실행정보를 로깅하는 간단한 미들웨어 """
    print("Agent 작업 시작...")

    # 계속해서 에이전트를 실행
    await next(context)

    print("Agent 작업 종료!")

---

## 🤖 Agent 생성 및 미들웨어 추가

In [ ]:
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential

agent = AzureOpenAIChatClient(credential=AzureCliCredential()).as_agent(
    name="GreetingAgent",
    instructions="당신은 한국어로 답변하는, 도움이 되는 어시스턴트입니다. 🤖✨",
    middleware=[logging_agent_middleware]   # 여기에 미들웨어 추가
)

async def main():
    result = await agent.run("안녕하세요!")
    print(result.text)

await main()

---

## 🔧 Function 기반 미들웨어

에이전트가 Function 도구를 사용하는 경우, Function 기반 미들웨어를 적용하면 함수 호출도 가로챌 수 있습니다.

### 📋 FunctionInvocationContext 속성

Function 기반 미들웨어는 에이전트 내의 함수 호출을 가로챕니다. `FunctionInvocationContext`를 사용하며, 다음 속성을 제공합니다:

| 속성 | 설명 |
|------|------|
| **function** | 호출되는 함수 |
| **arguments** | 함수에 대한 검증된 인수 |
| **metadata** | 미들웨어 간에 추가 데이터를 저장하기 위한 딕셔너리 |
| **result** | 함수의 반환 값 (수정 가능) |
| **terminate** | 추가 처리를 중지하는 플래그 |
| **kwargs** | 이 함수를 호출한 채팅 메서드에 전달되는 추가 키워드 인수 |  


> **💡 팁**  
> `next` 호출은 다음 미들웨어로 넘기거나, 실제 함수를 실행합니다.

In [ ]:
from agent_framework import FunctionInvocationContext

def get_time():
    """현재 시간을 가져옵니다."""
    from datetime import datetime
    return datetime.now().strftime("%H:%M:%S")

async def logging_function_middleware(
    context: FunctionInvocationContext,
    next: Callable[[FunctionInvocationContext], Awaitable[None]],
) -> None:
    """함수 호출을 기록하는 미들웨어입니다."""
    print(f"호출한 function : {context.function.name}")

    await next(context)

    print(f"function 결과: {context.result}")

# Add both the function and middleware to your agent
agent = AzureOpenAIChatClient(credential=AzureCliCredential()).as_agent(
    name="TimeAgent",
    instructions="현재 시간을 알려주세요.",
    tools=[get_time],
    middleware=[logging_function_middleware],
) 
async def main():
    result = await agent.run("현재 시간이 어떻게 되나요?")
    print(result.text)

await main()

---

## 🎯 Run 수준에서 미들웨어 사용

Agent 수준이 아닌, 특정 Run 수준에만 미들웨어를 추가할 수도 있습니다.

In [ ]:
agent = AzureOpenAIChatClient(credential=AzureCliCredential()).as_agent(
    name="TimeAgent",
    instructions="당신은 현재 시간을 알려주는 에이전트입니다."
) 

async def main():

    # Use middleware for this specific run only
    result = await agent.run(
        "좋은 하루 되세요!",
        middleware=[logging_agent_middleware]
    )
    print("미들웨어 동작 결과 : " + result.text)
    print("=" * 70)
    result = await agent.run("무엇을 알려주실 수 있나요?")
    print("일반 결과 : " + result.text)

await main()